<a href="https://colab.research.google.com/github/agdev/SDS-CP008-superstore-customer-segmentation/blob/main/notebooks/alex/SDS_CP008_superstore_customer_segmentation_Notebook_2_Clustering_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pydantic-settings -qU
# !pip install pandas-profiling[notebook] -qU
!pip install --upgrade pandas-profiling -qU

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.2/356.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 15.5 MB/s eta 0:00:00


# **Importing libraries**

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

import plotly
import plotly.express as px
import plotly.graph_objs as gobj
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
from plotly import figure_factory as FF
from pprint import pprint
import warnings
import os
warnings.filterwarnings('ignore')

In [3]:
# print('os.curdir', os.curdir)
print('os.getcwd()', os.getcwd())
print('os.listdir()', os.listdir())

os.getcwd() /content
os.listdir() ['.config', 'sample_data']


# **Uploading Files**

In [4]:
from google.colab import files

import io

if not os.path.exists('Sample - Superstore.csv'):
  uploaded = files.upload()
  df = pd.read_csv(io.BytesIO(uploaded['Sample - Superstore.csv']), encoding='latin1')
else:
  df = pd.read_csv(r'/content/Sample - Superstore.csv', encoding='latin1')
df.head()

Saving Sample - Superstore.csv to Sample - Superstore.csv


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


# **Current Features/Variables**

In [5]:
for col in df.columns:
  print(col)

Row ID
Order ID
Order Date
Ship Date
Ship Mode
Customer ID
Customer Name
Segment
Country
City
State
Postal Code
Region
Product ID
Category
Sub-Category
Product Name
Sales
Quantity
Discount
Profit


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [7]:
df.describe()

,Row ID,Postal Code,Sales,Quantity,Discount,Profit
count,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000,9994.000000
mean,4997.500000,55190.379428,229.858001,3.789574,0.156203,28.656896
std,2885.163629,32063.693350,623.245101,2.225110,0.206452,234.260108
min,1.000000,1040.000000,0.444000,1.000000,0.000000,-6599.978000
25%,2499.250000,23223.000000,17.280000,2.000000,0.000000,1.728750
50%,4997.500000,56430.500000,54.490000,3.000000,0.200000,8.666500
75%,7495.750000,90008.000000,209.940000,5.000000,0.200000,29.364000
max,9994.000000,99301.000000,22638.480000,14.000000,0.800000,8399.976000


In [8]:
print('The dataset has {} rows and {} columns.'.format(df.shape[0],df.shape[1]))

The dataset has 9994 rows and 21 columns.


# **Converting Date columns to date type**

In [9]:
df['Order Date'] = pd.to_datetime(df['Order Date'],format='mixed')
df['Ship Date']  = pd.to_datetime(df['Ship Date'],format='mixed')

# **Unique values per column**

In [10]:
print(df.nunique())

Row ID           9994
Order ID         5009
Order Date       1237
Ship Date        1334
Ship Mode           4
Customer ID       793
Customer Name     793
Segment             3
Country             1
City              531
State              49
Postal Code       631
Region              4
Product ID       1862
Category            3
Sub-Category       17
Product Name     1850
Sales            5825
Quantity           14
Discount           12
Profit           7287
dtype: int64


In [11]:
df = df.drop(['Row ID', 'Customer Name', 'Country'], axis=1)
df.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Segment,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Corporate,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [12]:
df['Category'].unique()

array(['Furniture', 'Office Supplies', 'Technology'], dtype=object)

In [13]:
df['Sub-Category'].unique()

array(['Bookcases', 'Chairs', 'Labels', 'Tables', 'Storage',
       'Furnishings', 'Art', 'Phones', 'Binders', 'Appliances', 'Paper',
       'Accessories', 'Envelopes', 'Fasteners', 'Supplies', 'Machines',
       'Copiers'], dtype=object)

# **Orders by location**

In [14]:
orders_by_loc = df.groupby(['Region','State','Postal Code'])['Order ID'].nunique().reset_index(name='N_Orders');
display(orders_by_loc)

,Region,State,Postal Code,N_Orders
0,Central,Illinois,60004,1
1,Central,Illinois,60016,2
2,Central,Illinois,60025,1
3,Central,Illinois,60035,3
4,Central,Illinois,60067,1
...,...,...,...,...
626,West,Washington,98632,2
627,West,Washington,98661,3
628,West,Washington,99207,5
629,West,Washington,99301,2


In [15]:
orders_by_loc['All'] = 'All'

fig_neg = px.treemap(orders_by_loc,path=['All','Region','State','Postal Code'],values='N_Orders',width=950,height=700);
fig_neg.update_layout(title_text='Total Number of Orders by Location',title_x=0.5,title_font=dict(size=25));
fig_neg.update_layout(margin=dict(t=50,l=25,r=25,b=25),paper_bgcolor="black",font=dict(color="white",size=18));
fig_neg.data[0].textinfo = 'label+value+percent parent'

plt.tight_layout();
fig_neg.show();

<Figure size 640x480 with 0 Axes>

# **Profit by Location**

In [16]:
profit_by_loc = df.groupby(['Region','State','Postal Code'])['Profit'].sum().reset_index();
display(profit_by_loc)

,Region,State,Postal Code,Profit
0,Central,Illinois,60004,1.2348
1,Central,Illinois,60016,-236.7674
2,Central,Illinois,60025,13.8579
3,Central,Illinois,60035,-169.9406
4,Central,Illinois,60067,23.2624
...,...,...,...,...
626,West,Washington,98632,38.8983
627,West,Washington,98661,75.4546
628,West,Washington,99207,172.2261
629,West,Washington,99301,357.5244


In [17]:
# Split the data into positive and negative profit dataframes
pos_profit = profit_by_loc[profit_by_loc['Profit'] > 0]
neg_profit = profit_by_loc[profit_by_loc['Profit'] < 0]

In [18]:
pos_profit['All'] = 'All'

fig = px.treemap(pos_profit, path=['All','Region','State','Postal Code'], values='Profit', width=950, height=700);
fig.update_layout(title_text='Positive Profit by Location',title_x=0.5,title_font=dict(size=25));
fig.update_layout(margin=dict(t=50,l=25,r=25,b=25),paper_bgcolor="black",font=dict(color="white",size=18));
fig.data[0].textinfo = 'label+value+percent parent'
# fig.data[0].hoverinfo = 'label+value+percent parent'
# Customize the tooltip format using hovertemplate
fig.data[0].hovertemplate = (
    'Location: %{label}<br>'   # Display the label (e.g., Postal Code or State)
    'Profit: $%{value:,}<br>'  # Display the value with formatting for currency
    'Percent of parent: %{percentParent:.2%}<br>'  # Show the percentage contribution
    '<extra></extra>'  # Remove the extra trace information
)

# plt.tight_layout();
fig.show();

In [19]:
display(neg_profit)

,Region,State,Postal Code,Profit
1,Central,Illinois,60016,-236.7674
3,Central,Illinois,60035,-169.9406
5,Central,Illinois,60068,-238.4500
6,Central,Illinois,60076,-346.8700
7,Central,Illinois,60089,-89.6478
...,...,...,...,...
568,West,Colorado,81001,-900.9719
595,West,Oregon,97224,-216.2861
596,West,Oregon,97301,-262.3311
598,West,Oregon,97477,-853.3086


In [20]:
# Calculate the absolute values of the 'Profit' column
neg_profit_abs = neg_profit.copy()
neg_profit_abs['Profit'] = neg_profit['Profit'].abs()

In [21]:
neg_profit_abs['All'] = 'All'

fig_neg = px.treemap(neg_profit_abs, path=['All','Region','State','Postal Code'], values='Profit', width=950, height=700);
fig_neg.update_layout(title_text='Loss by Location',title_x=0.5,title_font=dict(size=25));
fig_neg.update_layout(margin=dict(t=50,l=25,r=25,b=25),paper_bgcolor="black",font=dict(color="white",size=18));
fig_neg.data[0].textinfo = 'label+value+percent parent'
# fig.data[0].hoverinfo = 'label+value+percent parent'
# Customize the tooltip format using hovertemplate
fig_neg.data[0].hovertemplate = (
    'Location: %{label}<br>'   # Display the label (e.g., Postal Code or State)
    'Profit: $%{value:,}<br>'  # Display the value with formatting for currency
    'Percent of parent: %{percentParent:.2%}<br>'  # Show the percentage contribution
    '<extra></extra>'  # Remove the extra trace information
)

# plt.tight_layout();
fig_neg.show();

# **Profit by Category and Sub-Category**

In [22]:
profit_by_categ = df.groupby(['Category','Sub-Category'])['Profit'].sum().reset_index();
display(profit_by_categ)

,Category,Sub-Category,Profit
0,Furniture,Bookcases,-3472.5560
1,Furniture,Chairs,26590.1663
2,Furniture,Furnishings,13059.1436
3,Furniture,Tables,-17725.4811
4,Office Supplies,Appliances,18138.0054
5,Office Supplies,Art,6527.7870
6,Office Supplies,Binders,30221.7633
7,Office Supplies,Envelopes,6964.1767
8,Office Supplies,Fasteners,949.5182
9,Office Supplies,Labels,5546.2540


In [23]:
# Split the data into positive and negative profit dataframes
pos_categ_profit = profit_by_categ[profit_by_categ['Profit'] > 0]
neg_categ_profit = profit_by_categ[profit_by_categ['Profit'] < 0]

In [24]:
pos_categ_profit['All'] = 'All'

fig = px.treemap(pos_categ_profit,path=['All','Category','Sub-Category'],values='Profit',width=950,height=700);
fig.update_layout(title_text='Profit by Category',title_x=0.5,title_font=dict(size=25));
fig.update_layout(margin=dict(t=50,l=25,r=25,b=25),paper_bgcolor="black",font=dict(color="white",size=18));
fig.data[0].textinfo = 'label+value+percent parent'
# fig.data[0].hoverinfo = 'label+value+percent parent'
# Customize the tooltip format using hovertemplate
fig.data[0].hovertemplate = (
    'Category: %{label}<br>'   # Display the label (e.g., Postal Code or State)
    'Profit: $%{value:,}<br>'  # Display the value with formatting for currency
    'Percent of parent: %{percentParent:.2%}<br>'  # Show the percentage contribution
    '<extra></extra>'  # Remove the extra trace information
)

# plt.tight_layout();
fig.show();

In [25]:
display(neg_categ_profit)

,Category,Sub-Category,Profit
0,Furniture,Bookcases,-3472.5560
3,Furniture,Tables,-17725.4811
12,Office Supplies,Supplies,-1189.0995


In [26]:
display(pos_categ_profit)

,Category,Sub-Category,Profit,All
1,Furniture,Chairs,26590.1663,All
2,Furniture,Furnishings,13059.1436,All
4,Office Supplies,Appliances,18138.0054,All
5,Office Supplies,Art,6527.7870,All
6,Office Supplies,Binders,30221.7633,All
7,Office Supplies,Envelopes,6964.1767,All
8,Office Supplies,Fasteners,949.5182,All
9,Office Supplies,Labels,5546.2540,All
10,Office Supplies,Paper,34053.5693,All
11,Office Supplies,Storage,21278.8264,All


In [27]:
# Calculate the absolute values of the 'Profit' column
neg_categ_profit_abs = neg_categ_profit.copy()
neg_categ_profit_abs['Profit'] = neg_categ_profit['Profit'].abs()
display(neg_categ_profit_abs)

,Category,Sub-Category,Profit
0,Furniture,Bookcases,3472.5560
3,Furniture,Tables,17725.4811
12,Office Supplies,Supplies,1189.0995


In [ ]:
neg_categ_profit_abs['All'] = 'All'

fig_neg = px.treemap(neg_categ_profit_abs,path=['All','Category','Sub-Category'],values='Profit',width=950,height=700);
fig_neg.update_layout(title_text='Loss by Category',title_x=0.5,title_font=dict(size=25));
fig_neg.update_layout(margin=dict(t=50,l=25,r=25,b=25),paper_bgcolor="black",font=dict(color="white",size=18));
fig_neg.data[0].textinfo = 'label+value+percent parent'
# fig.data[0].hoverinfo = 'label+value+percent parent'
# Customize the tooltip format using hovertemplate
fig_neg.data[0].hovertemplate = (
    'Category: %{label}<br>'   # Display the label (e.g., Postal Code or State)
    'Profit: $%{value:,}<br>'  # Display the value with formatting for currency
    'Percent of parent: %{percentParent:.2%}<br>'  # Show the percentage contribution
    '<extra></extra>'  # Remove the extra trace information
)

# plt.tight_layout();
fig_neg.show();

# **Calculating LTV of customers**

In [33]:
df_clv = df[['Customer ID', 'Discount', 'Sales', 'Profit','Order Date', 'Ship Date','Order ID','Segment']]
print(df_clv.head())

  Customer ID  Discount     Sales    Profit Order Date  Ship Date  \
0    CG-12520      0.00  261.9600   41.9136 2016-11-08 2016-11-11   
1    CG-12520      0.00  731.9400  219.5820 2016-11-08 2016-11-11   
2    DV-13045      0.00   14.6200    6.8714 2016-06-12 2016-06-16   
3    SO-20335      0.45  957.5775 -383.0310 2015-10-11 2015-10-18   
4    SO-20335      0.20   22.3680    2.5164 2015-10-11 2015-10-18   

         Order ID    Segment  
0  CA-2016-152156   Consumer  
1  CA-2016-152156   Consumer  
2  CA-2016-138688  Corporate  
3  US-2015-108966   Consumer  
4  US-2015-108966   Consumer  


# **Creating a new Feature "Lifespan"**

In [29]:
# Calculate the lifespan for each customer
customer_lifespan = df_clv.groupby('Customer ID').agg({'Order Date': ['min', 'max']})
customer_lifespan.columns = ['FirstOrder', 'LastOrder']

# Calculate lifespan in days
customer_lifespan['LifespanDays'] = (customer_lifespan['LastOrder'] - customer_lifespan['FirstOrder']).dt.days

# Optionally convert lifespan to years
customer_lifespan['LifespanYears'] = customer_lifespan['LifespanDays'] / 365

print(customer_lifespan)

            FirstOrder  LastOrder  LifespanDays  LifespanYears
Customer ID                                                   
AA-10315    2014-03-31 2017-06-29          1186       3.249315
AA-10375    2014-04-21 2017-12-11          1330       3.643836
AA-10480    2014-05-04 2017-04-15          1077       2.950685
AA-10645    2014-06-22 2017-11-05          1232       3.375342
AB-10015    2014-02-18 2016-11-10           996       2.728767
...                ...        ...           ...            ...
XP-21865    2014-01-20 2017-11-17          1397       3.827397
YC-21895    2014-11-17 2017-12-26          1135       3.109589
YS-21880    2015-01-12 2017-12-21          1074       2.942466
ZC-21910    2014-10-13 2017-11-06          1120       3.068493
ZD-21925    2014-08-27 2017-06-11          1019       2.791781

[793 rows x 4 columns]


In [38]:
df_clv = pd.merge(df_clv, customer_lifespan, on='Customer ID', how='left')
df_clv.head()

,Customer ID,Discount,Sales,Profit,Order Date,Ship Date,Order ID,Segment,FirstOrder,LastOrder,LifespanDays,LifespanYears
0,CG-12520,0.00,261.9600,41.9136,2016-11-08,2016-11-11,CA-2016-152156,Consumer,2015-10-15,2017-01-26,469,1.284932
1,CG-12520,0.00,731.9400,219.5820,2016-11-08,2016-11-11,CA-2016-152156,Consumer,2015-10-15,2017-01-26,469,1.284932
2,DV-13045,0.00,14.6200,6.8714,2016-06-12,2016-06-16,CA-2016-138688,Corporate,2016-06-12,2017-12-11,547,1.498630
3,SO-20335,0.45,957.5775,-383.0310,2015-10-11,2015-10-18,US-2015-108966,Consumer,2015-10-11,2017-12-01,782,2.142466
4,SO-20335,0.20,22.3680,2.5164,2015-10-11,2015-10-18,US-2015-108966,Consumer,2015-10-11,2017-12-01,782,2.142466


In [39]:
# Calculate total sales for each customer
customer_sales = df_clv.groupby('Customer ID')['Sales'].sum()

# Calculate average order value (AOV)
aov = customer_sales / df_clv.groupby('Customer ID').size()

# Assume a purchase frequency (PF) and average customer lifespan (ACL)
pf = df_clv.groupby('Customer ID').size().mean()
acl = df_clv.groupby('Customer ID')['LifespanYears'].mean()
print('========aov========')
print(aov)
print('========pf========')
print(pf)
print('========acl========')
print(acl)
# Calculate CLV
clv = aov * pf * acl

# Create a new DataFrame with CLV
clv_df = pd.DataFrame({'Customer ID': clv.index, 'CLV': clv.values})
print('========clv_df========')
clv_df.head()

========aov========
Customer ID
AA-10315    505.778182
AA-10375     70.426000
AA-10480    149.209333
AA-10645    282.607500
AB-10015    147.692667
               ...    
XP-21865     84.809214
YC-21895    681.793750
YS-21880    560.037000
ZC-21910    258.893774
ZD-21925    165.993778
Length: 793, dtype: float64
========pf========
12.602774274905423
========acl========
Customer ID
AA-10315    3.249315
AA-10375    3.643836
AA-10480    2.950685
AA-10645    3.375342
AB-10015    2.728767
              ...   
XP-21865    3.827397
YC-21895    3.109589
YS-21880    2.942466
ZC-21910    3.068493
ZD-21925    2.791781
Name: LifespanYears, Length: 793, dtype: float64
========clv_df========


,Customer ID,CLV
0,AA-10315,20711.810944
1,AA-10375,3234.133602
2,AA-10480,5548.620046
3,AA-10645,12021.749781
4,AB-10015,5079.156139


# **Segmentation by Discount**

In [40]:
# Select Variables for Discount dataframe
df_discount = df[['Customer ID', 'Sales', 'Profit','Order Date', 'Discount','Segment','Region']]
print(df_discount.head())


  Customer ID     Sales    Profit Order Date  Discount    Segment Region
0    CG-12520  261.9600   41.9136 2016-11-08      0.00   Consumer  South
1    CG-12520  731.9400  219.5820 2016-11-08      0.00   Consumer  South
2    DV-13045   14.6200    6.8714 2016-06-12      0.00  Corporate   West
3    SO-20335  957.5775 -383.0310 2015-10-11      0.45   Consumer  South
4    SO-20335   22.3680    2.5164 2015-10-11      0.20   Consumer  South


In [41]:
# merging lifespan into dataframe
df_discount = pd.merge(df_discount, customer_lifespan, on='Customer ID', how='left')
df_discount.head()

,Customer ID,Sales,Profit,Order Date,Discount,Segment,Region,FirstOrder,LastOrder,LifespanDays,LifespanYears
0,CG-12520,261.9600,41.9136,2016-11-08,0.00,Consumer,South,2015-10-15,2017-01-26,469,1.284932
1,CG-12520,731.9400,219.5820,2016-11-08,0.00,Consumer,South,2015-10-15,2017-01-26,469,1.284932
2,DV-13045,14.6200,6.8714,2016-06-12,0.00,Corporate,West,2016-06-12,2017-12-11,547,1.498630
3,SO-20335,957.5775,-383.0310,2015-10-11,0.45,Consumer,South,2015-10-11,2017-12-01,782,2.142466
4,SO-20335,22.3680,2.5164,2015-10-11,0.20,Consumer,South,2015-10-11,2017-12-01,782,2.142466


# **Merge Discount and LTV**

In [42]:
# merging CLV into dataframe
df_clv_discount = pd.merge(df_discount, clv_df, on='Customer ID', how='left')
df_clv_discount.head()

,Customer ID,Sales,Profit,Order Date,Discount,Segment,Region,FirstOrder,LastOrder,LifespanDays,LifespanYears,CLV
0,CG-12520,261.9600,41.9136,2016-11-08,0.00,Consumer,South,2015-10-15,2017-01-26,469,1.284932,3720.600137
1,CG-12520,731.9400,219.5820,2016-11-08,0.00,Consumer,South,2015-10-15,2017-01-26,469,1.284932,3720.600137
2,DV-13045,14.6200,6.8714,2016-06-12,0.00,Corporate,West,2016-06-12,2017-12-11,547,1.498630,2349.284499
3,SO-20335,957.5775,-383.0310,2015-10-11,0.45,Consumer,South,2015-10-11,2017-12-01,782,2.142466,4684.811536
4,SO-20335,22.3680,2.5164,2015-10-11,0.20,Consumer,South,2015-10-11,2017-12-01,782,2.142466,4684.811536


# **CLV vs Discount**

In [46]:
import plotly.express as px

fig = px.scatter(df_clv_discount, x="Discount", y="CLV", color="Segment",
                 title="Customer Lifetime Value vs Discount with Segment Differentiation")


fig.show()

# **Lifespan vs . Discount**

In [47]:
import plotly.express as px

fig = px.scatter(df_clv_discount, x="Discount", y="LifespanDays", color="Segment",
                 title="Customer Lifespan Days vs Discount with Segment Differentiation")


fig.show()

# **Lifespan vs. CLV**

In [51]:
import plotly.express as px

fig = px.scatter(df_clv_discount, y="CLV", x="LifespanDays", color="Segment",
                 title="Customer Lifespan Days vs Customer LV with Segment Differentiation")


fig.show()

# **Segmentation Observations**

From the charts above we can see that discounts do not affect **CLV** or **Life span**
We do see that the longer Life span is the greater CLV is
The Segmentation lense depends on what we are trying to achieve
So for identifying worthy or more attention needed customers  I would look at CLV, Lifespan and Profit
For Marketing and/or Support Region and Profit and maybe even on zipcode level